# Pre-requisite installations if needed

In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install oauth2client

# ------------

In [77]:
import argparse
import pandas as pd
import json
import os
from pathlib import Path

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
from helper_functions import initialize_analyticsreporting, get_report, print_response, VIEW_ID

analytics = initialize_analyticsreporting()

In [78]:
# Date format in 'YYYY-MM-DD'
# You can also use relative dates for simplicity
# start_date = "50daysAgo"
# end_date = "today"
# end_date = "yesterday"
start_date = "2021-01-01"
end_date = "2021-04-21"

### Get a report of all the events that occured with a status of either "success" or "error"

In [79]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:totalEvents'}],
        'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
both_data, success_data, error_data = [], [], []

for res in response_rows:
    both_cell_data = [res["dimensions"][0], res["dimensions"][1], res["metrics"][0]["values"][0]]
    both_data.append(both_cell_data)
    if res["dimensions"][0] == "Success":
        success_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        success_data.append(success_cell_data)
    if res["dimensions"][0] == "Error":
        error_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        error_data.append(error_cell_data)

folder_path = os.path.join("result_csv", "status_count")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(both_data, columns = ['Status', 'Action', 'Values'])
result_path = os.path.join(folder_path, "Both-" + start_date + ":" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(success_data, columns = ['Action', 'Values'])
result_path = os.path.join(folder_path, "Success-" + start_date + ":" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(error_data, columns = ['Action', 'Values'])
result_path = os.path.join(folder_path, "Error-" + start_date + ":" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all app launches

In [80]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:totalEvents'}],
        'dimensions': [{'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
app_launch_os, app_launch_soda = [], []

for res in response_rows:
    if res["dimensions"][0] == "App Launched - OS":
        app_launch_os_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        app_launch_os.append(app_launch_os_cell_data)
    if res["dimensions"][0] == "App Launched - SODA":
        app_launch_soda_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        app_launch_soda.append(app_launch_soda_cell_data)

folder_path = os.path.join("result_csv", "app_launched")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(app_launch_os, columns = ['OS', 'Values'])
result_path = os.path.join(folder_path, "os-" + start_date + ":" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(app_launch_soda, columns = ['SODA Version', 'Values'])
result_path = os.path.join(folder_path, "soda_version-" + start_date + ":" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all users

In [86]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:users'}],
        'dimensions': [{'name': 'ga:userType'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
data = []

for res in response_rows:
    cell_data = [res["dimensions"][0], res["metrics"][0]["values"][0]]
    data.append(cell_data)
    
folder_path = os.path.join("result_csv", "users")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(data, columns = ['Type', 'Values'])
result_path = os.path.join(folder_path, "users-" + start_date + ":" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

{'dimensions': ['New Visitor'], 'metrics': [{'values': ['18']}]}
{'dimensions': ['Returning Visitor'], 'metrics': [{'values': ['22']}]}


In [99]:
def number_of_actions(start_date, end_date, category, action):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []

    for res in response_rows:
        if res["dimensions"][0] == category:
            if res["dimensions"][1] == action:
                cell_data = [res["dimensions"][2], res["metrics"][0]["values"][0]]
                data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Dataset_name', 'Values'])
    result_path = os.path.join(folder_path, action + "-" + start_date + ":" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting the names of datasets where an action is applicable
## all responses go to the custom folder
# number_of_actions(start_date, end_date, <type>, <action_name>)
number_of_actions(start_date, end_date, "Success", "Manage Dataset - Create Empty Dataset")

In [98]:
def dataset_actions(start_date, end_date, dataset_name):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []

    for res in response_rows:
        if res["dimensions"][2].find(dataset_name) != -1 or res["dimensions"][1].find(dataset_name) != -1:
            cell_data = [res["dimensions"][0], res["dimensions"][1], res["metrics"][0]["values"][0]]
            data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Action', 'Values'])
    result_path = os.path.join(folder_path, dataset_name + "(actions)-" + start_date + ":" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the actions for a specific dataset
## all responses go to the custom folder
# dataset_actions(start_date, end_date, <Dataset_name>)
dataset_actions(start_date, end_date, "test-ps-SODA")

In [118]:
def dataset_statistics(start_date, end_date):
            
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:uniqueEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    
    for res in response_rows:
#         print(res)
        if res["dimensions"][1] == "Upload Local Dataset - Number of Files":
            cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            data.append(cell_data)
        if res["dimensions"][1] == "Upload Local Dataset - size":
            cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
        if res["dimensions"][1] == "Generate Dataset - Number of Files":
            cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            data.append(cell_data)
        if res["dimensions"][1] == "Generate Dataset - Size":
            cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Number of Files', 'Size in (bytes)'])
    result_path = os.path.join(folder_path, "dataset_statistics-" + start_date + ":" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the details for a specific dataset
## all responses go to the custom folder
# num_of_files_folders_in_dataset(start_date, end_date, <Dataset_name>)
dataset_statistics(start_date, end_date)

# Not complete - Upload local is ok. Generate dataset needs more work for the name

In [115]:
def num_of_files_folders_in_dataset(start_date, end_date, dataset_name):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []
    
    for res in response_rows:
        if res["dimensions"][1].find(dataset_name) != -1:
            if res["dimensions"][1].find('Number of Files') != -1:
                cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            if res["dimensions"][1].find('- size') != -1:
                cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
            
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:eventValue'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    
    for res in response_rows:
        print(res)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Number of Files', 'Size in (bytes)'])
    result_path = os.path.join(folder_path, dataset_name + "(details)-" + start_date + ":" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the details for a specific dataset
## all responses go to the custom folder
# num_of_files_folders_in_dataset(start_date, end_date, <Dataset_name>)
num_of_files_folders_in_dataset(start_date, end_date, "test-ps-SODA")

{'dimensions': ['Error', 'Generate Dataset - - Number of Files', '(not set)'], 'metrics': [{'values': ['1']}]}
{'dimensions': ['Error', 'Generate Dataset - - Size', '(not set)'], 'metrics': [{'values': ['1']}]}
{'dimensions': ['Error', 'Generate Dataset - Number of Files', '(not set)'], 'metrics': [{'values': ['1']}]}
{'dimensions': ['Error', 'Generate Dataset - Size', '(not set)'], 'metrics': [{'values': ['1']}]}
{'dimensions': ['Success', 'Generate Dataset - - Number of Files', '(not set)'], 'metrics': [{'values': ['49']}]}
{'dimensions': ['Success', 'Generate Dataset - - Size', '(not set)'], 'metrics': [{'values': ['1462173']}]}
{'dimensions': ['Success', 'Generate Dataset - Local - Number of Files', '1-6'], 'metrics': [{'values': ['62']}]}
{'dimensions': ['Success', 'Generate Dataset - Local - Number of Files', '7-12'], 'metrics': [{'values': ['70']}]}
{'dimensions': ['Success', 'Generate Dataset - Local - Size', '1-6'], 'metrics': [{'values': ['547271731']}]}
{'dimensions': ['Succ